Работу выполнил Данил Исламов (Stepik ID: 274397404)

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Задание 3

## Классификация текстов

В этом задании вам предстоит попробовать несколько методов, используемых в задаче классификации, а также понять насколько хорошо модель понимает смысл слов и какие слова в примере влияют на результат.

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import f1_score

from torchtext.legacy import datasets

from torchtext.legacy.data import Field, LabelField, BucketIterator

from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.autonotebook import tqdm

В этом задании мы будем использовать библиотеку torchtext. Она довольна проста в использовании и поможет нам сконцентрироваться на задаче, а не на написании Dataloader'а.

In [ ]:
TEXT = Field(sequential=True, lower=True, include_lengths=True)  # Поле текста
LABEL = LabelField(dtype=torch.float)  # Поле метки

In [ ]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Датасет, на котором мы будем проводить эксперименты, это комментарии к фильмам c сайта IMDB.

In [ ]:
train, test = datasets.IMDB.splits(TEXT, LABEL)  # загрузим датасет
train, valid = train.split(random_state=random.seed(SEED))  # разобьем на части

In [ ]:
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, valid_iter, test_iter = BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    sort_within_batch = True,
    device = device)

## RNN

Для начала попробуем использовать рекурентные нейронные сети. На семинаре вы познакомились с GRU, вы можете также попробовать LSTM. Можно использовать для классификации как hidden_state, так и output последнего токена.

In [ ]:
# Будем использовать реализацию LSTM из PyTorch
class RNNBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.n_directions = int(bidirectional) + 1

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.rnn = nn.LSTM(input_size=embedding_dim,
                          hidden_size=hidden_dim,
                          num_layers=n_layers,
                          bidirectional=bidirectional,
                          dropout=dropout)
        
        self.dropout = nn.Dropout(dropout)

        self.fc = nn.Linear(in_features=hidden_dim * self.n_directions, 
                            out_features=output_dim)
        
        
    def forward(self, text, text_lengths):
        
        embedded = self.embedding(text)

        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)  
        
        if bidirectional:
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        else:
            hidden = hidden[-1]
        
        hidden = self.dropout(hidden)
            
        return self.fc(hidden)

Поиграйтесь с гиперпараметрами

In [ ]:
vocab_size = len(TEXT.vocab)
emb_dim = 100
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
patience = 3
max_grad_norm = 2

In [ ]:
model = RNNBaseline(
    vocab_size=vocab_size,
    embedding_dim=emb_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX
)

In [ ]:
model = model.to(device)

In [ ]:
opt = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

max_epochs = 20

Обучите сетку! Используйте любые вам удобные инструменты, Catalyst, PyTorch Lightning или свои велосипеды.

In [ ]:
import numpy as np

min_loss = np.inf
cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        
        opt.zero_grad()
        
        text = batch.text[0].to(device)
        text_lengths = batch.text[1].cpu()
        labels = batch.label.to(device)
        
        prediction = model(text, text_lengths).view(-1)
        
        loss = loss_func(prediction, labels)
        train_loss += loss.item()
        loss.backward()
        
        if max_grad_norm is not None:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        opt.step()

    train_loss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        with torch.no_grad():
            text = batch.text[0].to(device)
            text_lengths = batch.text[1].cpu()
            labels = batch.label.to(device)
        
            prediction = model(text, text_lengths).view(-1)

            loss = loss_func(prediction, labels)
            val_loss += loss.item()

    val_loss /= len(valid_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

Epoch: 1, Training Loss: 0.6564365259922333, Validation Loss: 0.6124367729081945


Epoch: 2, Training Loss: 0.549890542008581, Validation Loss: 0.6779991004426601


Epoch: 3, Training Loss: 0.39292103046700905, Validation Loss: 0.4375696508056026


Epoch: 4, Training Loss: 0.2668639061714176, Validation Loss: 0.4201506214626765


Epoch: 5, Training Loss: 0.1730647550990982, Validation Loss: 0.46753702931484936


<All keys matched successfully>

Посчитайте f1-score вашего классификатора на тестовом датасете (непосредственно код вычисления приведён ниже).

**Ответ**: 0.8299

In [ ]:
model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Epoch {epoch}")
preds = []
truth = []
for it, batch in pbar:
    with torch.no_grad():
        text = batch.text[0].to(device)
        text_lengths = batch.text[1].cpu()
        truth = np.concatenate((truth, batch.label.cpu().numpy()))
        
        prediction = np.around(F.sigmoid(model(text, text_lengths).view(-1).cpu()))
        preds = np.concatenate((preds, prediction))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
f1_score(truth, preds)

0.8298924731182795

## CNN

![](https://www.researchgate.net/publication/333752473/figure/fig1/AS:769346934673412@1560438011375/Standard-CNN-on-text-classification.png)

Для классификации текстов также часто используют сверточные нейронные сети. Идея в том, что, как правило, сентимент содержат словосочетания из двух-трех слов, например "очень хороший фильм" или "невероятная скука". Проходясь сверткой по этим словам мы получим какой-то большой скор и выхватим его с помощью MaxPool. Далее идет обычная полносвязная сетка. Важный момент: свертки применяются не последовательно, а параллельно. Давайте попробуем!

In [ ]:
TEXT = Field(sequential=True, lower=True, batch_first=True)  # batch_first, т.к. мы используем conv  
LABEL = LabelField(batch_first=True, dtype=torch.float)

train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

TEXT.build_vocab(trn)
LABEL.build_vocab(trn)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key=lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

Вы можете использовать Conv2d с `in_channels=1, kernel_size=(kernel_sizes[0], emb_dim)` или Conv1d c `in_channels=emb_dim, kernel_size=kernel_sizes[0]`. Но хорошенько подумайте над shape в обоих случаях.

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, emb_dim, out_channels, kernel_sizes, dropout=0.5,):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.conv_0 = nn.Conv1d(in_channels=emb_dim, out_channels=out_channels, 
                                kernel_size=kernel_sizes[0])
        
        self.conv_1 = nn.Conv1d(in_channels=emb_dim, out_channels=out_channels, 
                                kernel_size=kernel_sizes[1])
        
        self.conv_2 = nn.Conv1d(in_channels=emb_dim, out_channels=out_channels, 
                                kernel_size=kernel_sizes[2])
        
        self.fc = nn.Linear(len(kernel_sizes) * out_channels, 1)
        
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, text):
           
        embedded = self.embedding(text)
        embedded = embedded.permute(0, 2, 1)

        conved_0 = F.relu(self.conv_0(embedded)) 
        conved_1 = F.relu(self.conv_1(embedded))
        conved_2 = F.relu(self.conv_2(embedded))
   
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))
            
        return self.fc(cat)

In [ ]:
kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
out_channels = 64
dropout = 0.2
dim = 300
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
patience = 3
max_grad_norm = 2

model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=out_channels,
            kernel_sizes=kernel_sizes, dropout=dropout)

In [ ]:
model.to(device)

CNN(
  (embedding): Embedding(202268, 300)
  (conv_0): Conv1d(300, 64, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(300, 64, kernel_size=(4,), stride=(1,))
  (conv_2): Conv1d(300, 64, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=192, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
opt = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

In [ ]:
max_epochs = 30

Обучите!

In [ ]:
import numpy as np

min_loss = np.inf
cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 

        opt.zero_grad()
        
        text = batch.text.to(device)
        labels = batch.label.to(device)
        
        prediction = model(text).view(-1)
        
        loss = loss_func(prediction, labels)
        train_loss += loss.item()
        loss.backward()
        
        if max_grad_norm is not None:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        opt.step()

    train_loss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        with torch.no_grad():
            text = batch.text.to(device)
            labels = batch.label.to(device)

            prediction = model(text).view(-1)

            loss = loss_func(prediction, labels)
            val_loss += loss.item()

    val_loss /= len(val_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

Epoch: 1, Training Loss: 0.5666555203225491, Validation Loss: 0.47581180234750114


Epoch: 2, Training Loss: 0.4000695310804966, Validation Loss: 0.38274052341779075


Epoch: 3, Training Loss: 0.2987266130691027, Validation Loss: 0.36558828751246136


Epoch: 4, Training Loss: 0.21193112226298255, Validation Loss: 0.33296910524368284


Epoch: 5, Training Loss: 0.14869946094542524, Validation Loss: 0.3303002566099167


Epoch: 6, Training Loss: 0.09933719669815398, Validation Loss: 0.33640106171369555


Epoch: 7, Training Loss: 0.06963090091454287, Validation Loss: 0.3429237926999728


<All keys matched successfully>

Посчитайте f1-score вашего классификатора (непосредственно код вычисления приведён ниже).

**Ответ**: 0.8703

In [ ]:
model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Epoch {epoch}")
preds = []
truth = []
for it, batch in pbar:
    with torch.no_grad():
        text = batch.text.to(device)
        truth = np.concatenate((truth, batch.label.cpu().numpy()))
        
        prediction = np.around(F.sigmoid(model(text).view(-1).cpu()))
        preds = np.concatenate((preds, prediction))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
f1_score(truth, preds)

0.8703167743725965

## Интерпретируемость

Посмотрим, куда смотрит наша модель. Достаточно запустить код ниже.

In [ ]:
!pip install -q captum

     |████████████████████████████████| 4.4MB 9.9MB/s 


In [ ]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)

In [ ]:
def forward_with_softmax(inp):
    logits = model(inp)
    return torch.softmax(logits, 0)[0][1]

def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))


# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    model.eval()
    text = [tok for tok in TEXT.tokenize(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=5000, return_convergence_delta=True)

    print('pred: ', LABEL.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            LABEL.vocab.itos[pred_ind],
                            LABEL.vocab.itos[label],
                            LABEL.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [ ]:
interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'Fascinating work', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)
interpret_sentence(model, 'Whole trash', label=0)

pred:  pos ( 0.99 ) , delta:  tensor([2.5049e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.60 ) , delta:  tensor([8.7813e-06], device='cuda:0', dtype=torch.float64)
pred:  pos ( 1.00 ) , delta:  tensor([3.4497e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.98 ) , delta:  tensor([9.4717e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.11 ) , delta:  tensor([6.8853e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.20 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.87 ) , delta:  tensor([0.0001], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.68 ) , delta:  tensor([3.6706e-05], device='cuda:0', dtype=torch.float64)


Попробуйте добавить свои примеры!

In [ ]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.99),pos,0.91,It was a fantastic performance ! pad
pos,pos (0.60),pos,-0.41,Best film ever pad pad pad pad
pos,pos (1.00),pos,1.06,Such a great show! pad pad pad
pos,pos (0.98),pos,1.40,Fascinating work pad pad pad pad pad
neg,neg (0.11),pos,-0.66,It was a horrible movie pad pad
neg,neg (0.20),pos,-0.51,I've never watched something as bad pad
neg,pos (0.87),pos,0.30,It is a disgusting movie! pad pad
neg,pos (0.68),pos,-0.16,Whole trash pad pad pad pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.99),pos,0.91,It was a fantastic performance ! pad
pos,pos (0.60),pos,-0.41,Best film ever pad pad pad pad
pos,pos (1.00),pos,1.06,Such a great show! pad pad pad
pos,pos (0.98),pos,1.40,Fascinating work pad pad pad pad pad
neg,neg (0.11),pos,-0.66,It was a horrible movie pad pad
neg,neg (0.20),pos,-0.51,I've never watched something as bad pad
neg,pos (0.87),pos,0.30,It is a disgusting movie! pad pad
neg,pos (0.68),pos,-0.16,Whole trash pad pad pad pad pad


## Эмбэдинги слов

Вы ведь не забыли, как мы можем применить знания о word2vec и GloVe. Давайте попробуем!

In [ ]:
TEXT = Field(sequential=True, lower=True, batch_first=True)
LABEL = LabelField(batch_first=True, dtype=torch.float)

train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

TEXT.build_vocab(trn, vectors=GloVe(name="6B", dim=300))
LABEL.build_vocab(trn)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

In [ ]:
kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
out_channels = 64
dropout = 0.05
dim = 300
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
patience = 3
max_grad_norm = 2

model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=out_channels,
            kernel_sizes=kernel_sizes, dropout=dropout)
model.to(device)

CNN(
  (embedding): Embedding(202268, 300)
  (conv_0): Conv1d(300, 64, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(300, 64, kernel_size=(4,), stride=(1,))
  (conv_2): Conv1d(300, 64, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=192, out_features=1, bias=True)
  (dropout): Dropout(p=0.05, inplace=False)
)

In [ ]:
word_embeddings = TEXT.vocab.vectors

prev_shape = model.embedding.weight.shape

model.embedding.weight = nn.Parameter(word_embeddings) # инициализируйте эмбэдинги

assert prev_shape == model.embedding.weight.shape
model.to(device)

opt = torch.optim.Adam(model.parameters())

Вы знаете, что делать.

In [ ]:
opt = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

max_epochs = 30

In [ ]:
# Функция из семинара, "замораживающая" эмбеддинги слов, чтобы не испортить 
# предобученные эмбеддинги на первых эпохах обучения
def freeze_embeddings(model, req_grad=False):
    embeddings = model.embedding
    for c_p in embeddings.parameters():
        c_p.requires_grad = req_grad

In [ ]:
import numpy as np

min_loss = np.inf
cur_patience = 0

freeze_embeddings(model)
for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    num_iter = 0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 

        opt.zero_grad()
        
        text = batch.text.to(device)
        labels = batch.label.to(device)
        
        prediction = model(text).view(-1)
        
        loss = loss_func(prediction, labels)
        train_loss += loss.item()
        loss.backward()
        
        if max_grad_norm is not None:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        opt.step()

    if epoch == 1:
        freeze_embeddings(model, True)

    train_loss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        
        with torch.no_grad():
            text = batch.text.to(device)
            labels = batch.label.to(device)

            prediction = model(text).view(-1)

            loss = loss_func(prediction, labels)
            val_loss += loss.item()

    val_loss /= len(val_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

Epoch: 1, Training Loss: 0.46973322038232845, Validation Loss: 0.3591162274281184


Epoch: 2, Training Loss: 0.29788079096453035, Validation Loss: 0.300602159400781


Epoch: 3, Training Loss: 0.16345088983321712, Validation Loss: 0.29256962090730665


Epoch: 4, Training Loss: 0.06499222058286197, Validation Loss: 0.2884636784593264


Epoch: 5, Training Loss: 0.0193102468675288, Validation Loss: 0.30650046865145364


Epoch: 6, Training Loss: 0.007957594727512694, Validation Loss: 0.322690649330616


<All keys matched successfully>

Посчитайте f1-score вашего классификатора.

**Ответ**: 0.8795

In [ ]:
model.eval()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
pbar.set_description(f"Epoch {epoch}")
preds = []
truth = []
for it, batch in pbar:
    with torch.no_grad():
        text = batch.text.to(device)
        truth = np.concatenate((truth, batch.label.cpu().numpy()))
        
        prediction = np.around(F.sigmoid(model(text).view(-1).cpu()))
        preds = np.concatenate((preds, prediction))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
f1_score(truth, preds)

0.8794843730998418

Проверим насколько все хорошо!

In [ ]:
PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)
vis_data_records_ig = []

interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'Fascinating work', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)
interpret_sentence(model, 'Whole trash', label=0)

pred:  pos ( 1.00 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.05 ) , delta:  tensor([5.6921e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.52 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.07 ) , delta:  tensor([4.0539e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([7.7601e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.05 ) , delta:  tensor([7.0260e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([0.0003], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.01 ) , delta:  tensor([3.7783e-05], device='cuda:0', dtype=torch.float64)


In [ ]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (1.00),pos,1.49,It was a fantastic performance ! pad
pos,neg (0.05),pos,-0.97,Best film ever pad pad pad pad
pos,pos (0.52),pos,1.27,Such a great show! pad pad pad
pos,neg (0.07),pos,-0.52,Fascinating work pad pad pad pad pad
neg,neg (0.00),pos,-0.84,It was a horrible movie pad pad
neg,neg (0.05),pos,-0.28,I've never watched something as bad pad
neg,neg (0.00),pos,-0.74,It is a disgusting movie! pad pad
neg,neg (0.01),pos,-1.06,Whole trash pad pad pad pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (1.00),pos,1.49,It was a fantastic performance ! pad
pos,neg (0.05),pos,-0.97,Best film ever pad pad pad pad
pos,pos (0.52),pos,1.27,Such a great show! pad pad pad
pos,neg (0.07),pos,-0.52,Fascinating work pad pad pad pad pad
neg,neg (0.00),pos,-0.84,It was a horrible movie pad pad
neg,neg (0.05),pos,-0.28,I've never watched something as bad pad
neg,neg (0.00),pos,-0.74,It is a disgusting movie! pad pad
neg,neg (0.01),pos,-1.06,Whole trash pad pad pad pad pad


# Итоги:

Модели показали следующие величины f1-score на тестовом датасете:

*   RNN: 0.8299
*   CNN: 0.8703
*   CNN + GloVe embeddings: 0.8795

Таким образом, лучшее (если сравнивать по данной метрике) качество показали свёрточные сети, при этом самый высокий результат в проведённых экспериментах показала свёрточная сеть, использовавшая предобученные эмбеддинги слов.

При визуализации того, на какие слова "обращают внимание" разные реализации CNN в искусственных примерах, было заметно, что модели не очень точны не только в итоговых предсказаниях, но и в определении слов, которые кажутся наиболее значимыми в конкретных предложениях. Так, например, сети выделяли такие слова как "film" или "movie" как несущие определённую эмоциональную окраску, что ощущается явно неверным. Слово же "best", к примеру, наоборот, не имело большого веса для определения смысла. Предположу, что это было связано непосредственно с датасетом, на котором обучалась модель — возможно, слово "best" в нём почти не встречалось, a "film" или "movie" были скорее характерны для текстов конкретного класса. Тем не менее, хотелось бы, чтобы результаты получались более общими — возможно, для этого стоило бы расширить датасет, например, комментариями к фильмам с других сайтов.

Таким образом, хоть f1-score у всех сетей получился вполне приемлемым, что означает умение моделей распознавать определённые закономерности в текстах конкретного датасета, их качество пока далеко не идеально, что и проявилось при тестировании на искусственных примерах. Для улучшения результатов, думаю, будет полезным использовать больше данных для обучения сети; возможно также, существуют какие-то более продвинутые подходы к обучению или подходящие для подобных задач архитектуры сетей, которые позволили бы добиться более высоких результатов. В любом случае, видно, что потенциал для улучшения явно есть.